# LMM hypotheses tests and confidence intervals

Often statistical models are used in order to determine which of the predictor variables have a significant relationship with the response variable. `LMM` has a number of methods to aid with this kind of statistical inference.

Below we will fit a linear mixed model using the Ruby gem [mixed\_models](https://github.com/agisga/mixed_models), and demostrate various types of hypotheses tests and confidence intervals available for objects of class `LMM`.


## Contents

* Data and linear mixed model

* Likelihood ratio test
  - Chi squared p-value
  - Bootstrap p-value

* Fixed effects hypotheses tests
  - Likelihood ratio test
  - Bootstrap test
  - Variances and covariances
  - Wald Z test

* Fixed effects confidence intervals
  - Bootstrap confidence intervals
  - Wald Z confidence intervals
  - All types of intervals at once

* Random effects hypotheses tests
  - Likelihood ratio test
  - Bootstrap test

## Data and linear mixed model

We use the same data and model formulation as in a [previous example](http://nbviewer.ipython.org/github/agisga/mixed_models/blob/master/notebooks/LMM_model_fitting.ipynb), where we have looked at variaous parameter estimates, and have shown that the model fit is good.

The data set, which is simulated, contains two numeric variables *Age* and *Aggression*, and two categorical variables *Location* and *Species*. These data are available for 100 (human and alien) individuals.

We model the *Aggression* level of an individual as a linear function of the *Age* (*Aggression* decreases with *Age*), with a different constant added for each *Species* (i.e. each species has a different base level of aggression). Moreover, we assume that there is a random fluctuation in *Aggression* due to the *Location* that an individual is at. Additionally, there is a random fluctuation in how *Age* affects *Aggression* at each different *Location*. 

Thus, the *Aggression* level of an individual of *Species* $spcs$ who is at the *Location* $lctn$ can be expressed as:
$$Aggression = \beta_{0} + \gamma_{spcs} + Age \cdot \beta_{1} + b_{lctn,0} + Age \cdot b_{lctn,1} + \epsilon,$$
where $\epsilon$ is a random residual, and the random vector $(b_{lctn,0}, b_{lctn,1})^T$ follows a multivariate normal distribution (the same distribution but different realizations of the random vector for each *Location*). That is, we have a linear mixed model with fixed effects $\beta_{0}, \beta_{1}, \gamma_{Dalek}, \gamma_{Ood}, \dots$, and random effects $b_{Asylum,0}, b_{Asylum,1}, b_{Earth,0},\dots$.

We fit the model with the convenient method `LMM#from_formula`, which mimics the behaviour of the function `lmer` from the `R` package `lme4`.

In [1]:
require 'mixed_models'

alien_species = Daru::DataFrame.from_csv '../examples/data/alien_species.csv'
model_fit = LMM.from_formula(formula: "Aggression ~ Age + Species + (Age | Location)", 
                             data: alien_species)

puts "Fixed effects terms estimates and some diagnostics:"
puts model_fit.fix_ef_summary.inspect(20)
puts "Random effects correlation structure:"
puts model_fit.ran_ef_summary.inspect(12)

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"http://d3js.org/d3.v3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t

Fixed effects terms estimates and some diagnostics:

#<Daru::DataFrame:70173242364960 @name = e70d6b20-176f-42d1-b96e-09e258d4e91d @size = 5>
                                     coef                   sd              z_score        WaldZ_p_value 
           intercept   1016.2867207696772    60.19727495932258   16.882603431075875                  0.0 
                 Age -0.06531615343467667  0.08988486367253856  -0.7266646548258817   0.4674314106158888 
   Species_lvl_Human    -499.693695290209   0.2682523406941927  -1862.7747813759402                  0.0 
     Species_lvl_Ood   -899.5693213535769   0.2814470814004366  -3196.2289922406044                  0.0 
Species_lvl_WeepingA  -199.58895804200762   0.2757835779525997   -723.7158917283754                  0.0 

Random effects correlation structure:

#<Daru::DataFrame:70173242286120 @name = 1346e45f-61ca-4603-8994-acf13a7231c3 @size = 2>
                 Location Location_Age 
    Location 104.26376362 -0.059863903 
Location_Age 

## Likelihood ratio test

Given two nested models, the `LMM.likelihood_ratio_test` tests whether the restricted simpler model is adequate. In this context 'nested' means that all predictors used in the restricted model must also be predictors in the full model (i.e. one model is a reduced version of the other more complex model). This method works only if both models were fit using the deviance (as opposed to REML criterion) as the objective function for the minimization (i.e. fit with `reml: false`). `LMM.likelihood_ratio_test` returns the p-value of the test.

Two methods are available to compute the p-value: approximation by a Chi squared distribution as delineated in section 2.4.1 in Pinheiro & Bates "Mixed Effects Models in S and S-PLUS" (2000), and a method based on bootstrapping as delineated in section 4.2.3 in Davison & Hinkley "Bootstrap Methods and their Application" (1997).

For example we can test the model formulation as above against a simpler model, which assumes that *Age* is neither a fixed effect nor a random effect. We can compute a likelihood ratio using the method `LMM.likelihood_ratio`.

In [2]:
complex_model = LMM.from_formula(formula: "Aggression ~ Age + Species + (Age | Location)", 
                                 data: alien_species, reml: false)
simple_model = LMM.from_formula(formula: "Aggression ~ Species + (1 | Location)", 
                                data: alien_species, reml: false)

LMM.likelihood_ratio(simple_model, complex_model)

454.36066138776386

### Chi squared p-value

We perform the likelihood ratio test using the method `LMM.likelihood_ratio_test` with `method: :chi2` to use the Chi squared approximation for the p-values.

In [3]:
chi2_p_value = LMM.likelihood_ratio_test(simple_model, complex_model, method: :chi2)

3.693825760409898e-98

The p-value is tiny, which implies that *Age* is a significant predictor of *Aggression*, and that the more complex model should be prefered.

### Bootstrap p-value

However, often one may not be sure whether the assumptions required for the validity of the Chi squared test are satisfied. In that case, one can compute a p-value with the bootstrap method by specifying `method: :bootstrap`, which by default uses all available CPUs in parallel.

In [4]:
bootstrap_p_value = LMM.likelihood_ratio_test(simple_model, complex_model, method: :bootstrap, nsim: 1000)

0.000999000999000999

Even though the p-value is not as extreme as with the Chi squared test, it still shows significance of the variable *Age*.

## Fixed effects hypotheses tests

Significance tests for the fixed effects can be performed with `LMM#fix_ef_p` (or its alias `LMM#fix_ef_test`). For a given fixed effects coefficient estimate the tested null hypothesis is that the true value of the coefficient is zero (i.e. no linear relationship to the response). 

That is, for the above model formulation we carry out hypotheses tests for each fixed effects terms $\beta_{i}$ or $\gamma_{species}$, testing the null $H_{0} : \beta_{i} = 0$ against the alternative $H_{a} : \beta_{i} \neq 0$, or respectively the null $H_{0} : \gamma_{species} = 0$ against the alternative $H_{a} : \gamma_{species} \neq 0$.

`LMM` currently offers three methods of hypotheses testing for fixed effects: *Wald Z test*, *likelihood ratio test*, and a *bootstrap test*. For a good discussion of the validity of different methods see [this entry from the wiki of the r-sig-mixed-models mailing list](http://glmm.wikidot.com/faq). Moreover, due to the equivalence of hypotheses tests and confidence intervals, an additional hypothesis testing tool are bootstrap confidence intervals which are described in a different section below.

### Likelihood ratio test for the fixed effects

The likelihood ratio test for fixed effects is actually merely a convenience method. It is a convenient interface to `LMM.likelihood_ratio_test` with `method: :chi2` described above. For example, we can test whether the fixed effects term *Age* is a significant predictor of *Aggression* in `complex_model` as follows.

In [5]:
lrt_p_value = complex_model.fix_ef_p(variable: :Age, method: :lrt)

0.4017669962430499

We see that *Age* does not seem be significant as a fixed effects term, even though it is in general a significant predictor as we have seen before.

### Bootstrap test for the fixed effects

Like the likelihood ratio test, the bootstrap test is merely a convenient shortcut to `LMM.likelihood_ratio_test` with `method: :bootstrap`. We can test the significance of the predictor variable *Age* in `complex_model` with:

In [6]:
bootstrap_p_value = complex_model.fix_ef_p(variable: :Age, method: :bootstrap, nsim: 1000)

0.5454545454545454

This result confirms the conclusion based of `method: :lrt`.

### Variances and covariances of the fixed effects coefficient estimates

The covariance matrix of the fixed effects estimates is returned by `LMM#fix_ef_cov_mat`, and the standard deviations of the fixed effects coefficients are returned by `LMM#fix_ef_sd`.

In [7]:
model_fit.fix_ef_sd

{:intercept=>60.19727495932258, :Age=>0.08988486367253856, :Species_lvl_Human=>0.2682523406941927, :Species_lvl_Ood=>0.2814470814004366, :Species_lvl_WeepingAngel=>0.2757835779525997}

### Wald Z tests for the fixed effects

Based on the values returned by `LMM#fix_ef_sd`, the [Wald Z test statistics](https://en.wikipedia.org/wiki/Wald_test#Test_on_a_single_parameter) for all fixed effects coefficients are computed by the method `LMM#fix_ef_z`.

In [8]:
model_fit.fix_ef_z

{:intercept=>16.882603431075875, :Age=>-0.7266646548258817, :Species_lvl_Human=>-1862.7747813759402, :Species_lvl_Ood=>-3196.2289922406044, :Species_lvl_WeepingAngel=>-723.7158917283754}

Based on the above Wald Z test statistics, hypotheses tests for each fixed effects term can be carried out.
The corresponding (approximate) p-values are obtained with `LMM#fix_ef_p` as follows.

In [9]:
model_fit.fix_ef_p(method: :wald)

{:intercept=>0.0, :Age=>0.4674314106158888, :Species_lvl_Human=>0.0, :Species_lvl_Ood=>0.0, :Species_lvl_WeepingAngel=>0.0}

We see that *Aggression* of each *Species* is significantly different from the base level (which is the species *Dalek* in this model), while statistically we don't have enough evidence to conclude that *Age* of an individual is a good predictor of his/her/its aggression level (which agrees with the conclusion obtained above with `method: :lrt` and `method: :bootstrap`).

## Confidence intervals for the fixed effects

Confidence intervals for the fixed effects terms can be computed with the method `LMM#fix_ef_conf_int`. The following types of confidence intervals are available:

* Wald Z intervals
* Basic bootstrap intervals
* Bootstrap normal intervals
* Bootstrap percentile intervals
* Studentized bootstrap intervals

### Bootstrap confidence intervals for the fixed effects

A detailed description of the bootstrap methods available in `LMM#fix_ef_conf_int` is given in [this blog post](http://agisga.github.io/bootstap_confidence_intervals/).

For example we can compute the studentized bootstrap confidence intervals (also called bootstrap-t) with 98% coverage as follows.

In [12]:
bootstrap_t_intervals = model_fit.fix_ef_conf_int(level: 0.98, method: :bootstrap, 
                                                  boottype: :studentized, nsim: 1000)

{:intercept=>[887.0918096452608, 1158.067406554853], :Age=>[-0.2708105986572986, 0.1383216555736241], :Species_lvl_Human=>[-500.30330413054435, -499.0937790890286], :Species_lvl_Ood=>[-900.2077898494975, -898.9197999237398], :Species_lvl_WeepingAngel=>[-200.15929741314994, -198.95979978371057]}

We see that *Age* is the only fixed effects predictor whose confidence interval contains zero, which implies that it probably has little linear relationship as a fixed effect to the response variable *Aggression*.

### Wald Z confidence intervals for the fixed effects

We can use the Wald Z statistic to compute confidence intervals as well. For example 90% confidence intervals for each fixed effects coefficient estimate can be computed as follows.

In [13]:
conf_int = model_fit.fix_ef_conf_int(level: 0.9, method: :wald)

{:intercept=>[917.2710147202404, 1115.302426819114], :Age=>[-0.2131635974544904, 0.08253129058513706], :Species_lvl_Human=>[-500.1349311257381, -499.2524594546799], :Species_lvl_Ood=>[-900.0322606062133, -899.1063821009406], :Species_lvl_WeepingAngel=>[-200.04258166045662, -199.13533442355862]}

For greated visual clarity we can put the coefficient estimates and the confidence intervals into a `Daru::DataFrame`:

In [14]:
df = Daru::DataFrame.rows(conf_int.values, order: [:lower90, :upper90], index: model_fit.fix_ef_names)
df[:coef] = model_fit.fix_ef.values
df

,lower90,upper90,coef
intercept,917.2710147202404,1115.302426819114,1016.2867207696772
Age,-0.2131635974544904,0.08253129058513706,-0.06531615343467667
Species_lvl_Human,-500.1349311257381,-499.2524594546799,-499.693695290209
Species_lvl_Ood,-900.0322606062133,-899.1063821009406,-899.5693213535769
Species_lvl_WeepingAngel,-200.04258166045662,-199.13533442355862,-199.58895804200762


### All types of confidence intervals at once

With `method: :all`, `LMM#fix_ef_conf_int` returns a Daru::DataFrame containing the confidence intervals obtained by each of the available methods. The data frame can be printed in form of a nice looking table for inspection. 

For example for the alien species data we obtain all types of 95% confidence intervals with:

In [15]:
ci = model_fit.fix_ef_conf_int(method: :all, nsim: 1000)

,intercept,Age,Species_lvl_Human,Species_lvl_Ood,Species_lvl_WeepingAngel
wald_z,"[898.3022298819501, 1134.2712116574044]","[-0.24148724898814486, 0.11085494211879152]","[-500.2194602167382, -499.16793036367983]","[-900.1209474966757, -899.0176952104781]","[-200.12948392232232, -199.04843216169292]"
boot_basic,"[889.5341272383348, 1132.4423502839113]","[-0.23280775311774404, 0.10408983509595293]","[-500.2257509346596, -499.2090341090452]","[-900.1121372464709, -899.0468322837926]","[-200.12208087572265, -199.0242010808312]"
boot_norm,"[894.6858944623998, 1137.4579914122567]","[-0.2358815463919327, 0.1058785345670842]","[-500.2180279012096, -499.1854775510564]","[-900.1044315575966, -899.0358105540224]","[-200.12655724007533, -199.05392093938082]"
boot_t,"[889.5341272383348, 1132.4423502839113]","[-0.23280775311774404, 0.10408983509595293]","[-500.2257509346596, -499.2090341090452]","[-900.1121372464709, -899.0468322837926]","[-200.12208087572265, -199.0242010808312]"
boot_perc,"[900.1310912554433, 1143.0393143010197]","[-0.23472214196530627, 0.10217544624839071]","[-500.1783564713728, -499.16163964575844]","[-900.0918104233613, -899.026505460683]","[-200.15371500318403, -199.0558352082926]"


Since here we are dealing with data that was simulated according to the assumptions of the linear mixed model, all parameters end up approximately meeting the normality assumptions, and therefore all confidence interval methods turn out to be pretty much equivalent. Often when analyzing less ideal data, this will not be the case. Then it might be necessary to compare different types of confidence intervals in order to draw the right conclusions.

## Random effects hypotheses tests

We can test individual random effects for siginificance with the method `LMM#ran_ef_p` (or its alias `LMM#ran_ef_test`). It offers two methods, `:lrt` and `:bootstrap`. Both are in fact merely convenient interfaces to `LMM.likelihood_ratio_test` described above.

### Likelihood ratio test for individual random effects

The likelihood ratio test for random effects can only be performed if the model was fit with option `reml: false`. 

For example we can test the random intercept term (due to *Location*) of `complex_model` as follows.

In [16]:
complex_model.ran_ef_p(variable: :intercept, grouping: :Location, method: :lrt)

1.3846568414022321e-151

This suggests that the variability in *Aggression* is significantly influenced by the effect of *Location*.

### Bootstrap test for individual random effects

This test can also be performed only if the model was fit with`reml: false`.

We can test whether there is a significant random variation of the effect of *Age* due to *Location* using `LMM#ran_ef_p` with `method: :bootstrap` as follows.

In [17]:
complex_model.ran_ef_p(variable: :Age, grouping: :Location, method: :bootstrap, nsim: 1000)

0.000999000999000999

We see that in fact the change of *Age* due to *Location* is a significant random effect.